In [1]:
import torch
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
from transformers import Trainer, TrainingArguments
from datasets import load_dataset

# 1. DistilBERT 모델과 토크나이저 로딩
model_name = "distilbert-base-uncased"
tokenizer = DistilBertTokenizer.from_pretrained(model_name)
model = DistilBertForSequenceClassification.from_pretrained(model_name)

/home/patrick/Programming/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [2]:
# 2. 데이터셋 로딩 (예: AG News dataset)
dataset = load_dataset("ag_news")
train_dataset = dataset["train"]
test_dataset = dataset["test"]

In [3]:
print(train_dataset['label'])

[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 

In [ ]:
# 클래스 레이블 범위 확인
# DistilBERT 모델을 사용하는 경우, 레이블은 0부터 num_classes - 1까지이어야 합니다. 예를 들어, 4개의 클래스가 있다면 레이블 값은 0, 1, 2, 3이어야 합니다.
# 데이터셋에서 레이블 값이 올바른지 확인하세요. 예를 들어, AG News 데이터셋을 사용하고 있다면 레이블이 0부터 3까지 있어야 합니다. 만약 레이블 값이 4 이상이면 오류가 발생할 수 있습니다.

for x in train_dataset['label']:
    if x >= 4:
        print("over value")

In [ ]:
# label 열의 타입 확인
# 레이블 값이 int 타입이어야 하는데, 만약 문자열로 되어 있으면 nll_loss에서 오류가 발생할 수 있습니다. 레이블 타입이 숫자형인지 확인하고, 필요하면 int로 변환해 주세요.

train_dataset = train_dataset.map(lambda e: {'label': int(e['label'])})

Map: 100%|██████████| 120000/120000 [00:02<00:00, 43996.20 examples/s]


In [ ]:
# torch.cuda 관련 설정
# 때때로 CUDA 관련 오류가 발생하는 이유는 메모리 부족이거나 잘못된 GPU 설정 때문일 수 있습니다. 
# 오류 메시지에서 device-side assert triggered가 나타났을 때는, CUDA 설정이 잘못되었을 가능성도 있습니다. 
# 이를 해결하기 위해 CUDA_LAUNCH_BLOCKING=1을 설정하여 디버깅할 수 있습니다.

!export CUDA_LAUNCH_BLOCKING=1

In [ ]:
# Gradient Accumulation 설정
# gradient_accumulation_steps가 너무 높은 값으로 설정되어 있으면 메모리 문제나 다른 오류를 유발할 수 있습니다. 이 값을 낮추거나 조정해보세요.
# training_args

NameError: name 'training_args' is not defined

In [10]:
# 클래스 개수 확인
# 모델을 훈련할 때 num_labels 파라미터가 올바르게 설정되어 있는지 확인해 보세요. 
# 예를 들어, DistilBertForSequenceClassification 모델을 사용할 때, num_labels를 데이터셋의 클래스 개수에 맞게 설정해야 합니다.

model = DistilBertForSequenceClassification.from_pretrained(model_name, num_labels=4)  # 예시

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
# 3. 데이터셋 전처리 (토크나이징)
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

In [12]:
# 4. 전처리된 데이터셋
train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

Map: 100%|██████████| 7600/7600 [00:04<00:00, 1632.50 examples/s]


In [13]:
# 5. Trainer 설정
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    gradient_accumulation_steps=1,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

In [14]:
# 6. 훈련 시작
trainer.train()

Step,Training Loss
10,1.409600
20,1.394500
30,1.389100
40,1.389800
50,1.380500
60,1.359500
70,1.329200
80,1.297800
90,1.219900
100,1.129200


TrainOutput(global_step=45000, training_loss=0.18969604702072426, metrics={'train_runtime': 11036.761, 'train_samples_per_second': 32.618, 'train_steps_per_second': 4.077, 'total_flos': 4.768996442112e+16, 'train_loss': 0.18969604702072426, 'epoch': 3.0})

In [ ]:
# 최종 훈련된 모델 저장
trainer.save_model("./final_model")

# 이전 파라미터 설정에서 save_steps를 사용하여 일정 주기마다 모델 저장 가능
# training_args = TrainingArguments(
#     output_dir="./results",
#     num_train_epochs=3,
#     per_device_train_batch_size=8,
#     per_device_eval_batch_size=8,
#     warmup_steps=500,
#     weight_decay=0.01,
#     logging_dir="./logs",
#     logging_steps=10,
#     save_steps=1000,  # 1000 steps마다 모델을 저장
#     save_total_limit=3  # 최대 3개의 체크포인트만 저장
# )

In [2]:
# 7. 모델 평가
results = trainer.evaluate()

NameError: name 'trainer' is not defined

In [6]:
import torch
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification

# 저장된 모델 불러오기
model = DistilBertForSequenceClassification.from_pretrained("./final_model")
model_name = "distilbert-base-uncased"
tokenizer = DistilBertTokenizer.from_pretrained(model_name)

In [7]:
# 8. 모델을 사용한 예측
texts = ["The quick brown fox jumps over the lazy dog.", "I love machine learning."]
inputs = tokenizer(texts, padding=True, truncation=True, return_tensors="pt")
# with torch.no_grad():
#     outputs = model(**inputs)
#     predictions = torch.argmax(outputs.logits, dim=-1)

# print("Predictions: ", predictions)

In [8]:
# 모델을 GPU로 이동 (GPU가 있을 경우)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# 입력 데이터도 GPU로 변환
inputs = tokenizer(texts, padding=True, truncation=True, return_tensors="pt")
inputs = {key: value.to(device) for key, value in inputs.items()}  # GPU로 이동

# 뉴스 카테고리 매핑 (AG News 데이터셋 기준)
label_map = {
    0: "World",
    1: "Sports",
    2: "Business",
    3: "Science/Technology"
}

with torch.no_grad():
    outputs = model(**inputs)
    predictions = torch.argmax(outputs.logits, dim=-1)

# 예측된 숫자 값을 카테고리명으로 변환
predicted_labels = [label_map[pred.item()] for pred in predictions]

# 결과 출력
for text, category in zip(texts, predicted_labels):
    print(f"기사: {text}\n분류된 카테고리: {category}\n")

기사: The quick brown fox jumps over the lazy dog.
분류된 카테고리: Business

기사: I love machine learning.
분류된 카테고리: Science/Technology

